In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,514763
1,Huelva,Confirmados PDIA,73105
2,Huelva,Confirmados PDIA 14 días,6079
3,Huelva,Tasa PDIA 14 días,"1180,9318074531386"
4,Huelva,Confirmados PDIA 7 días,2034
5,Huelva,Tasa PDIA 7 dias,"395,13329435099257"
6,Huelva,Total Confirmados,73350
7,Huelva,Curados,56841
8,Huelva,Fallecidos,443


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  73105.0


/tmp/ipykernel_35508/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  21325.0


/tmp/ipykernel_35508/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_35508/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_35508/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_35508/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 246 personas en los últimos 7 días 

Un positivo PCR cada 82 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,73105.0,6079.0,2034.0,514763.0,395.133294,1180.931807,261.0
Huelva-Costa,42176.0,3034.0,973.0,290301.0,335.169359,1045.122132,127.0
Huelva (capital),21325.0,1730.0,580.0,142538.0,406.909035,1213.711431,78.0
Sierra de Huelva-Andévalo Central,9752.0,1819.0,656.0,67400.0,973.293769,2698.813056,72.0
Condado-Campiña,20706.0,1216.0,402.0,157062.0,255.949880,774.216551,61.0
Aljaraque,2898.0,253.0,78.0,21706.0,359.347646,1165.576338,14.0
Villalba del Alcor,597.0,33.0,30.0,3334.0,899.820036,989.802040,12.0
Valverde del Camino,2178.0,433.0,179.0,12714.0,1407.896807,3405.694510,12.0
Encinasola,239.0,35.0,20.0,1292.0,1547.987616,2708.978328,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,272.0,72.0,33.0,1682.0,1961.950059,4280.618312,10.0
Cortegana,740.0,179.0,65.0,4635.0,1402.373247,3861.920173,8.0
Marines (Los),44.0,15.0,5.0,410.0,1219.512195,3658.536585,0.0
Nerva,712.0,185.0,74.0,5127.0,1443.339185,3608.347962,4.0
Cerro de Andévalo (El),463.0,82.0,20.0,2320.0,862.068966,3534.482759,2.0
Valverde del Camino,2178.0,433.0,179.0,12714.0,1407.896807,3405.694510,12.0
Linares de la Sierra,24.0,9.0,3.0,268.0,1119.402985,3358.208955,0.0
Jabugo,308.0,74.0,27.0,2247.0,1201.602136,3293.279929,2.0
Puerto Moral,39.0,9.0,2.0,281.0,711.743772,3202.846975,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alájar,71.0,13.0,1.0,814.0,122.850123,1597.051597,0.0,0.076923
Almendro (El),89.0,9.0,1.0,835.0,119.760479,1077.844311,0.0,0.111111
Villanueva de los Castillejos,322.0,21.0,3.0,2849.0,105.300105,737.100737,0.0,0.142857
Fuenteheridos,83.0,14.0,2.0,751.0,266.311585,1864.181092,0.0,0.142857
Manzanilla,261.0,20.0,3.0,2152.0,139.405204,929.368030,0.0,0.150000
Bonares,624.0,34.0,6.0,6075.0,98.765432,559.670782,2.0,0.176471
Paterna del Campo,408.0,16.0,3.0,3478.0,86.256469,460.034503,0.0,0.187500
Almonte,3147.0,181.0,36.0,24577.0,146.478415,736.460919,3.0,0.198895
Villanueva de las Cruces,51.0,5.0,1.0,385.0,259.740260,1298.701299,0.0,0.200000
